In [1]:
import pandas as pd


In [2]:
# Import FRED look up csv
fred = pd.read_csv(
    '~/Downloads/FRED2_csv_2/README_TITLE_SORT.txt', 
    skiprows=9, 
    skipfooter=22, 
    sep=';',
    parse_dates=[5],
    skipinitialspace=True,
    engine='python'
)
fred.columns = fred.columns.str.strip()
# Remove all "(DISCONTINUED)" rows
fred = fred[~fred['Title'].str.contains('DISCONTINUED')]
fred = fred.set_index('Title')


In [3]:
# Get all U.S. population data
uspop = fred[fred.index.str.match('Population.*United States.*')]
# Remove % change units
uspop = uspop[~uspop['Units'].str.contains('%')]
# Find the latest
uspop = uspop.loc[uspop['Last Updated'].argmax()].iloc[0]
uspop

File                   P\O\P\T\O\POPTOTUSA647NWDB.csv          
Units                                                   Persons
Frequency                                                     A
Seasonal Adjustment                                         NSA
Last Updated                                2015-07-07 00:00:00
Name: Population, Total for United States, dtype: object

In [4]:
# Get all personal income/exspense data
personal = fred[fred.index.str.match('Personal.*')]

# Remove State based data
personal = personal[~personal.index.str.contains('.*, [A-Z]{2}')]
personal = personal[~personal.index.str.contains('.*for Social Insurance in.*')]
personal = personal[~personal.index.str.contains('.*District of Columbia.*')]

# Remove per capitas
personal = personal[~personal.index.str.contains('.*per capita.*')]

# Remove old 2012 style data
personal = personal[~personal.index.str.contains('.*,.* for United States')]

# Remove nonresidents
personal = personal[~personal.index.str.contains('.*nonresidents.*')]

# Remove % change and index units
personal = personal[~personal['Units'].str.contains('%|Index|Persons|National Currency')]

# Get only monthly, quartarly or yearly data
personal = personal[personal['Frequency'].str.contains('M|Q|A')]

# Remap frequency to number of months
personal['Frequency'].replace({'M':1, 'Q':3, 'A':12}, inplace=True)

# Keep only the highest resolution of duplicate data
personal = personal.reset_index() \
    .sort_values(['Title', 'Frequency', 'Last Updated'], ascending=[1, 1, 0]) \
    .drop_duplicates(subset='Title', keep='first') \
    .sort_values('Title').set_index('Title')


In [5]:
for p in personal.index:
    print p


Personal Consumption Expenditures
Personal Consumption Expenditures by Type of Product: Services: Household Consumption Expenditures: Housing
Personal Consumption Expenditures: Durable Goods
Personal Consumption Expenditures: Goods
Personal Consumption Expenditures: Nondurable Goods
Personal Consumption Expenditures: Services
Personal Current Tax Receipts: State and Local Government: Property Taxes
Personal Current Transfer Receipts
Personal Income
Personal Income Receipts on Assets
Personal Income Receipts on Assets: Personal Dividend Income
Personal Income Receipts on Assets: Personal Interest Income
Personal Income for United States
Personal Saving
Personal Saving for United States
Personal Tax and Nontax Payments for United States
Personal consumption expenditures excluding food and energy
Personal consumption expenditures: Accounting and other business services
Personal consumption expenditures: Admissions to specified spectator amusements
Personal consumption expenditures: Admiss